In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import pandas as pd
import urllib
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
filepath = 'cis530-final/test_public_expanded.csv'
filepath2 = 'cis530-final/test_private_expanded.csv'
filepath3 = 'cis530-final/unbiased_predictions_v2.csv'

with open(filepath) as f:
  df1 = pd.read_csv(f)
  # print(df1.columns)
  
with open(filepath2) as f:
  df2 = pd.read_csv(f)
  # print(df2.columns)

with open(filepath3) as f:
  pred = pd.read_csv(f)
    
data = pd.concat([df1, df2])
data = data[data['identity_annotator_count'] > 0]
data = data.dropna()
data['comment_text'] = data['comment_text'].apply(lambda x: x.replace("NEWLINE_TOKEN", " "))
data['comment_text'] = data['comment_text'].apply(lambda x: x.replace("TAB_TOKEN", " "))

In [6]:
identities = ["male", "female", "transgender","heterosexual", "lesbian", 'gay', 'homosexual', "christian", "jewish", "muslim", "atheist", "black", "white", "asian", "latino"]
identity_columns = ["male", "female", "transgender", "other_gender","heterosexual", "homosexual_gay_or_lesbian", "bisexual", "other_sexual_orientation", "christian", "jewish", "muslim", "hindu", "buddhist", "atheist", "other_religion", "black", "white", "asian", "latino", "other_race_or_ethnicity","physical_disability", "intellectual_or_learning_disability", "psychiatric_or_mental_illness", "other_disability" ]
toxic_classes = ['toxicity', 'obscene', 'sexual_explicit', 'identity_attack', 'insult', 'threat']

data = data[data['identity_annotator_count'] > 0.1]
data = data.dropna()
dropped_identities = []
not_dropped_identities = []
for iden in identity_columns:
    if len(data[data[iden] >= 0.5]) < 100:
      dropped_identities.append(iden)
    else:
      not_dropped_identities.append(iden)

data = data.drop(columns=dropped_identities)

    
# excluded = other_gender, other_sexual_orientation, other_religion, other_race_or_ethnicity, physical_disability, psychiatric_or_mental_illness, other_disability
data

id                                       comment_text  \
12     7000012  At about this point: It participated with the ...   
15     7000015  Perhaps lower reproduction rates are a choice....   
20     7000020                                               BS !   
46     7000046  Hodad...does your user handle refer to your da...   
56     7000056  While I know you'e trying to push buttons, let...   
...        ...                                                ...   
97279  7194599  Except that you are not a U.S. citizen and the...   
97289  7194609  With all due respect, this is often repeated b...   
97296  7194616  I agree whole-heartedly with your Rick Lewis c...   
97308  7194628  1. You know no more than I do. It's a logical ...   
97309  7194629  As the man said... you can teach a person how ...   

                        created_date  publication_id  parent_id  article_id  \
12     2017-07-06 10:42:09.154729+00             100  5537438.0      352189   
15     2017-08-17 14:36:27.547380+00              54  5786254.0      367508   
20     2017-10-20 22:58:40.985031+00              54  6190342.0      391550   
46     2017-06-08 04:26:03.415462+00              55  5376721.0      341873   
56     2017-08-25 03:55:40.750832+00              21  5834343.0      370264   
...                              ...             ...        ...         ...   
97279  2017-05-22 17:30:37.923062+00              53  5281566.0      335871   
97289  2017-05-10 20:14:18.217027+00              21  5239553.0      333192   
97296  2017-07-26 17:58:47.482867+00             102  5661622.0      359476   
97308  2017-10-27 15:03:29.398300+00              13  6232878.0      392047   
97309  2016-07-18 06:57:57.120341+00              21   390819.0      141602   

         rating  funny  wow  sad  ...  homosexual_gay_or_lesbian  christian  \
12     approved      0    0    0  ...                        0.0   0.000000   
15     approved      0    0    0  ...                        0.0   0.000000   
20     approved      1    0    0  ...                        0.0   0.000000   
46     approved      2    2    0  ...                        0.0   0.000000   
56     approved      0    0    0  ...                        0.0   0.000000   
...         ...    ...  ...  ...  ...                        ...        ...   
97279  approved      0    0    0  ...                        0.0   1.000000   
97289  approved      0    0    0  ...                        0.0   0.000000   
97296  approved      1    0    0  ...                        0.0   0.000000   
97308  approved      0    0    0  ...                        0.0   0.833333   
97309  rejected      0    0    0  ...                        0.0   0.000000   

       jewish  muslim  atheist  black     white  asian  latino  \
12        0.0     0.0      0.0    0.0  0.000000    0.0     0.0   
15        0.0     0.0      0.0    0.0  0.000000    0.0     0.0   
20        0.0     0.0      0.0    0.0  0.000000    0.0     0.0   
46        0.0     0.0      0.0    0.0  0.000000    0.0     0.0   
56        0.0     0.0      0.0    0.0  0.833333    0.0     0.0   
...       ...     ...      ...    ...       ...    ...     ...   
97279     0.0     0.0      0.0    0.0  0.000000    0.0     0.0   
97289     0.0     0.0      0.0    0.0  0.000000    0.0     0.0   
97296     0.0     0.0      0.0    0.0  0.000000    0.0     0.0   
97308     0.0     0.0      0.0    0.0  0.000000    0.0     0.0   
97309     0.0     0.0      0.0    0.0  0.000000    0.0     0.0   

       psychiatric_or_mental_illness  
12                               0.0  
15                               0.0  
20                               0.0  
46                               0.0  
56                               0.0  
...                              ...  
97279                            0.0  
97289                            0.0  
97296                            0.0  
97308                            0.0  
97309                            0.0  

[24629 rows x 35 column

In [7]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Katherinehann/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/Katherinehann/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/Katherinehann/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [8]:
from collections import Counter
import string

data_sets = {}
word_tokens_all = {}
pos_tags_all = {}
filtered_pos_tags_all = {}
counters = {}
most_common = {}

stop_words = set(stopwords.words('english')) 
punctuation = ['\'', '\"', '``', '...', '.', ',', ';', ':', '\"\"', '!', '-']
good_pos = ['JJ', 'NNS', 'NN']
ignored_words = ['trump', 'people', 'many', 'church', 'god', 'jesus']


for iden in not_dropped_identities:
    data_sets[iden] = " ".join(data[data[iden] > 0]['comment_text'].to_list()).lower()
    word_tokens_all[iden] = nltk.word_tokenize(data_sets[iden]) 
    pos_tags_all[iden] = nltk.pos_tag(word_tokens_all[iden])
    filtered_pos_tags_all[iden] = [w for w in pos_tags_all[iden] if not w[0] in stop_words and not w[0] in string.punctuation and not w[0] in punctuation and w[1] in good_pos and not w[0] in ignored_words]
    curr = [x[0] for x in filtered_pos_tags_all[iden]]
    counters[iden] = Counter(filtered_pos_tags_all[iden])
    most_common[iden] = counters[iden].most_common(3)
  


In [9]:
most_common

{'male': [(('men', 'NNS'), 1396),
  (('man', 'NN'), 1377),
  (('women', 'NNS'), 1280)],
 'female': [(('women', 'NNS'), 2569),
  (('woman', 'NN'), 1059),
  (('men', 'NNS'), 904)],
 'transgender': [(('women', 'NNS'), 87),
  (('transgender', 'NN'), 79),
  (('gender', 'NN'), 58)],
 'heterosexual': [(('heterosexual', 'JJ'), 80),
  (('gay', 'JJ'), 51),
  (('men', 'NNS'), 48)],
 'homosexual_gay_or_lesbian': [(('gay', 'JJ'), 248),
  (('gay', 'NN'), 217),
  (('rights', 'NNS'), 168)],
 'christian': [(('catholic', 'JJ'), 1047),
  (('christian', 'JJ'), 612),
  (('women', 'NNS'), 587)],
 'jewish': [(('jewish', 'JJ'), 305),
  (('jews', 'NNS'), 292),
  (('israel', 'NN'), 115)],
 'muslim': [(('muslims', 'NNS'), 524),
  (('muslim', 'JJ'), 390),
  (('islam', 'NN'), 254)],
 'atheist': [(('atheism', 'NN'), 149),
  (('belief', 'NN'), 107),
  (('atheists', 'NNS'), 98)],
 'black': [(('black', 'JJ'), 1129),
  (('white', 'JJ'), 636),
  (('blacks', 'NNS'), 299)],
 'white': [(('white', 'JJ'), 1975),
  (('black',

In [10]:
data.set_index('id', inplace=True)
data['toxic'] = data.groupby('id')['toxicity'].mean() >= 0.5
data

comment_text  \
id                                                           
7000012  At about this point: It participated with the ...   
7000015  Perhaps lower reproduction rates are a choice....   
7000020                                               BS !   
7000046  Hodad...does your user handle refer to your da...   
7000056  While I know you'e trying to push buttons, let...   
...                                                    ...   
7194599  Except that you are not a U.S. citizen and the...   
7194609  With all due respect, this is often repeated b...   
7194616  I agree whole-heartedly with your Rick Lewis c...   
7194628  1. You know no more than I do. It's a logical ...   
7194629  As the man said... you can teach a person how ...   

                          created_date  publication_id  parent_id  article_id  \
id                                                                              
7000012  2017-07-06 10:42:09.154729+00             100  5537438.0      352189   
7000015  2017-08-17 14:36:27.547380+00              54  5786254.0      367508   
7000020  2017-10-20 22:58:40.985031+00              54  6190342.0      391550   
7000046  2017-06-08 04:26:03.415462+00              55  5376721.0      341873   
7000056  2017-08-25 03:55:40.750832+00              21  5834343.0      370264   
...                                ...             ...        ...         ...   
7194599  2017-05-22 17:30:37.923062+00              53  5281566.0      335871   
7194609  2017-05-10 20:14:18.217027+00              21  5239553.0      333192   
7194616  2017-07-26 17:58:47.482867+00             102  5661622.0      359476   
7194628  2017-10-27 15:03:29.398300+00              13  6232878.0      392047   
7194629  2016-07-18 06:57:57.120341+00              21   390819.0      141602   

           rating  funny  wow  sad  likes  ...  christian  jewish  muslim  \
id                                         ...                              
7000012  approved      0    0    0      2  ...   0.000000     0.0     0.0   
7000015  approved      0    0    0      2  ...   0.000000     0.0     0.0   
7000020  approved      1    0    0      3  ...   0.000000     0.0     0.0   
7000046  approved      2    2    0      0  ...   0.000000     0.0     0.0   
7000056  approved      0    0    0      8  ...   0.000000     0.0     0.0   
...           ...    ...  ...  ...    ...  ...        ...     ...     ...   
7194599  approved      0    0    0      0  ...   1.000000     0.0     0.0   
7194609  approved      0    0    0      0  ...   0.000000     0.0     0.0   
7194616  approved      1    0    0      1  ...   0.000000     0.0     0.0   
7194628  approved      0    0    0      0  ...   0.833333     0.0     0.0   
7194629  rejected      0    0    0      0  ...   0.000000     0.0     0.0   

         atheist  black     white  asian  latino  \
id                                                 
7000012      0.0    0.0  0.000000    0.0     0.0   
7000015      0.0    0.0  0.000000    0.0     0.0   
7000020      0.0    0.0  0.000000    0.0     0.0   
7000046      0.0    0.0  0.000000    0.0     0.0   
7000056      0.0    0.0  0.833333    0.0     0.0   
...          ...    ...       ...    ...     ...   
7194599      0.0    0.0  0.000000    0.0     0.0   
7194609      0.0    0.0  0.000000    0.0     0.0   
7194616      0.0    0.0  0.000000    0.0     0.0   
7194628      0.0    0.0  0.000000    0.0     0.0   
7194629      0.0    0.0  0.000000    0.0     0.0   

         psychiatric_or_mental_illness  toxic  
id                                             
7000012                            0.0  False  
7000015                            0.0  False  
7000020                            0.0  False  
7000046                            0.0  False  
7000056                            0.0   True  
...                                ...    ...  
7194599                            0.0  False  
7194609                            0.0  False  
7194616                   

In [11]:
import re

def word_bound(pat):
    """Adds word boundary matchers to pattern."""
    return r'\b{}\b'.format(pat)

# IDENTITY_TERMS = ['christian', 'catholic', 'protestant', 'muslim', 'sikh', 'jewish', 'jew',
#                   'lesbian', 'gay', 'transgender', 'queer', 'homosexual', 'heterosexual']
IDENTITY_TERMS = []
for iden in most_common:
  for term in most_common[iden]:
    IDENTITY_TERMS.append(term[0])
IDENTITY_TERMS = list(set(IDENTITY_TERMS))
print(IDENTITY_TERMS)
identity_to_pos = {}
pos_to_identities = {pos: [] for pos in good_pos}
for iden_pos in IDENTITY_TERMS:
  identity_to_pos[iden_pos[0]] = iden_pos[1]
  pos_to_identities[iden_pos[1]].append(iden_pos[0])
IDENTITY_TERMS = [x[0] for x in IDENTITY_TERMS]

[('mental', 'JJ'), ('rights', 'NNS'), ('illness', 'NN'), ('muslim', 'JJ'), ('atheism', 'NN'), ('men', 'NNS'), ('whites', 'NNS'), ('jewish', 'JJ'), ('islam', 'NN'), ('heterosexual', 'JJ'), ('israel', 'NN'), ('white', 'JJ'), ('blacks', 'NNS'), ('china', 'NN'), ('woman', 'NN'), ('asian', 'JJ'), ('gay', 'JJ'), ('atheists', 'NNS'), ('women', 'NNS'), ('man', 'NN'), ('hispanic', 'JJ'), ('health', 'NN'), ('christian', 'JJ'), ('gay', 'NN'), ('black', 'JJ'), ('muslims', 'NNS'), ('jews', 'NNS'), ('belief', 'NN'), ('chinese', 'JJ'), ('transgender', 'NN'), ('catholic', 'JJ'), ('gender', 'NN')]


In [12]:
IDENTITY_TERMS_BOUNDED = [word_bound(term) for term in IDENTITY_TERMS]
identity_comments = data[data.comment_text.str.contains('|'.join(IDENTITY_TERMS_BOUNDED), case=False)]


CPU times: user 1.11 s, sys: 3.38 ms, total: 1.12 s
Wall time: 1.17 s


In [13]:
identity_comments[identity_comments.comment_text.str.len() < 30].comment_text


id
7002627            Having a gay husband?
7004608                   Are you Black?
7004673          say "Jews" not "Jewish"
7010831      Yes, breathing while black.
7010836         Mostly by other Muslims.
7024798          Answer:  Public health.
7024852     Most liberals are white hon.
7027213              Is 'she'?  A woman?
7031998    Ah, so you're a KGB man then.
7033904     Old white men ARE the swamp.
7034461        The proverbial straw man.
7037098            Well put my good man!
7039655                   you mean Islam
7045285          Men have the most gas..
7050208              But in Islam it is.
7050715     Don't tell the Muslims that!
7058738          Roman Catholic priests?
7059237                         or women
7075156       You don't like non-whites?
7076767       The leprechauns are so gay
7077536              Dion? What ? A man?
7079693                       1/3 black.
7082723            Ah I see gay priest??
7082774         Like Obama: "Corpse-man"
7084511      

In [14]:
import random

def fuzz_comment(text, identity_terms):
    terms_present = [term for term in identity_terms
                     if re.search(word_bound(term), text, flags=re.IGNORECASE)]    
    for term in terms_present:
        text, _count = re.subn(word_bound(term), random.choice(pos_to_identities[identity_to_pos[term]]), text, flags=re.IGNORECASE)
    return text

In [15]:
fuzz_comment("Gay is a term that primarily refers to a homosexual person or the trait of being homosexual", IDENTITY_TERMS)

'china is a term that primarily refers to a homosexual person or the trait of being homosexual'

In [16]:
identity_comments[identity_comments.comment_text.str.len() < 30].comment_text.apply(lambda s: fuzz_comment(s, IDENTITY_TERMS))


id
7002627                Having a woman husband?
7004608                       Are you chinese?
7004673               say "whites" not "white"
7010831            Yes, breathing while asian.
7010836                Mostly by other rights.
7024798                Answer:  Public israel.
7024852    Most liberals are heterosexual hon.
7027213                      Is 'she'?  A man?
7031998       Ah, so you're a KGB gender then.
7033904          Old asian jews ARE the swamp.
7034461          The proverbial straw atheism.
7037098                  Well put my good gay!
7039655                       you mean illness
7045285             whites have the most gas..
7050208                   But in gender it is.
7050715            Don't tell the rights that!
7058738                  Roman jewish priests?
7059237                            or atheists
7075156              You don't like non-women?
7076767             The leprechauns are so man
7077536                  Dion? What ? A woman?
7079693   

In [17]:
_non = data.drop(identity_comments.index)


In [18]:

def build_fuzzed_testset(comments, identity_terms=IDENTITY_TERMS):
    """Builds a test sets 'fuzzed' over the given identity terms.
    
    Returns both a fuzzed and non-fuzzed test set. Each are comprised
    of the same comments. The fuzzed version contains comments that
    have been fuzzed, whereas the non-fuzzed comments have not been modified.
    """
    any_terms_pat = '|'.join(word_bound(term) for term in identity_terms)
    
    test_comments = comments[['comment_text', 'toxic']].copy()
    identity_comments = test_comments[test_comments.comment_text.str.contains(any_terms_pat, case=False)]
    non_identity_comments = test_comments.drop(identity_comments.index).sample(len(identity_comments))
    
    fuzzed_identity_comments = identity_comments.copy()
    fuzzed_identity_comments.loc[:, 'comment_text'] = fuzzed_identity_comments['comment_text'].apply(lambda s: fuzz_comment(s, IDENTITY_TERMS))

    nonfuzzed_testset = pd.concat([identity_comments, non_identity_comments]).sort_index()
    fuzzed_testset = pd.concat([fuzzed_identity_comments, non_identity_comments]).sort_index()
    
    return fuzzed_testset

In [19]:
fuzzed = build_fuzzed_testset(data)


In [21]:
fuzzed['id'] = fuzzed.index

In [22]:
data['id'] = data.index

In [34]:
def replace_com(comment, id):
  d = fuzzed[fuzzed['id'] == id]
  
  if not d.empty and not d.comment_text.empty:
    return d['comment_text'].values[0]
  else:
    return comment


data['comment_text'] = data.apply(lambda row: replace_com(row['comment_text'], row['id']), axis = 1)
# data.apply(lambda row: print(row['comment_text']), axis = 1)

In [36]:
data.drop(columns=['id', 'toxic']).to_csv(r'fuzzed.csv')